IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M4\FAULT_M4_5


In [3]:
guasto=1

VOLO m4 FAULT 5%

In [4]:
rcou_m4_fault5 = pd.read_csv("RCOU.csv")
rcou_m4_fault5 = rcou_m4_fault5.astype("float64")
rcou_m4_fault5 = rcou_m4_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m4_fault5 = rcou_m4_fault5[((rcou_m4_fault5['C9'] >= potenza) & (rcou_m4_fault5['C10'] >= potenza) & (rcou_m4_fault5['C11'] >= potenza) & (rcou_m4_fault5['C12'] >= potenza) & (rcou_m4_fault5['C13'] >= potenza) & (rcou_m4_fault5['C14']>= potenza))]
rcou_m4_fault5=rcou_m4_fault5.reset_index(drop=True)
display(rcou_m4_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
0,121397337.0,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0
1,121410324.0,1439.0,1355.0,1377.0,1419.0,1418.0,1379.0
2,121424667.0,1408.0,1411.0,1412.0,1407.0,1409.0,1410.0
3,121445598.0,1499.0,1352.0,1382.0,1476.0,1455.0,1406.0
4,121456481.0,1474.0,1408.0,1412.0,1471.0,1444.0,1440.0
...,...,...,...,...,...,...,...
4788,178847914.0,1552.0,1434.0,1532.0,1458.0,1369.0,1601.0
4789,178858913.0,1555.0,1427.0,1555.0,1428.0,1410.0,1568.0
4790,178869630.0,1521.0,1463.0,1552.0,1427.0,1369.0,1596.0
4791,178879924.0,1514.0,1468.0,1544.0,1435.0,1355.0,1604.0


In [5]:
min=rcou_m4_fault5['TimeUS'][0]
max=rcou_m4_fault5['TimeUS'][len(rcou_m4_fault5)-1]
print(max)
print(min)

178933828.0
121397337.0


In [6]:
xkf1_m4_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m4_fault5 = xkf1_m4_fault5.astype("float64")
xkf1_m4_fault5 = xkf1_m4_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m4_fault5 = xkf1_m4_fault5[((xkf1_m4_fault5['TimeUS'] >= min) & (xkf1_m4_fault5['TimeUS'] <= max))]
xkf1_m4_fault5 = xkf1_m4_fault5.reset_index(drop=True)
print(xkf1_m4_fault5)

           TimeUS  Roll  Pitch    Yaw
0     121408104.0  1.06   0.85  11.75
1     121422491.0  1.01   0.87  11.74
2     121432861.0  1.10   0.86  11.75
3     121442909.0  1.16   0.87  11.74
4     121453711.0  1.13   0.88  11.75
...           ...   ...    ...    ...
5181  178889699.0 -0.12  -0.02   8.32
5182  178901435.0 -0.23  -0.05   8.15
5183  178911672.0 -0.31   0.00   8.00
5184  178921855.0 -0.40   0.11   7.82
5185  178931763.0 -0.43   0.34   7.65

[5186 rows x 4 columns]


In [7]:
att_m4_fault5 = pd.read_csv("ATT.csv")
att_m4_fault5 = att_m4_fault5.astype("float64")
att_m4_fault5 = att_m4_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m4_fault5 = att_m4_fault5[((att_m4_fault5['TimeUS'] >= min) & (att_m4_fault5['TimeUS'] <= max	))]
att_m4_fault5=att_m4_fault5.reset_index(drop=True)
print(att_m4_fault5)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     121409547.0     0.29  1.06     -0.18   0.85   11.69  11.75
1     121423615.0     0.29  1.01     -0.19   0.87   11.69  11.74
2     121434103.0     0.29  1.10     -0.20   0.86   11.69  11.75
3     121444886.0     0.29  1.16     -0.20   0.87   11.69  11.74
4     121455563.0     0.29  1.13     -0.21   0.88   11.69  11.75
...           ...      ...   ...       ...    ...     ...    ...
5181  178891074.0     2.47 -0.12     -0.14  -0.02    6.91   8.32
5182  178902591.0     2.50 -0.23     -0.22  -0.05    6.91   8.15
5183  178912882.0     2.54 -0.31     -0.32   0.00    6.90   8.00
5184  178922946.0     2.58 -0.40     -0.42   0.11    6.90   7.82
5185  178932994.0     2.62 -0.43     -0.51   0.34    6.90   7.65

[5186 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m4_fault5)>len(xkf1_m4_fault5):
    to_add=att_m4_fault5[len(xkf1_m4_fault5):]
    att_m4_fault5=att_m4_fault5[:len(xkf1_m4_fault5)]

if len(xkf1_m4_fault5)>len(att_m4_fault5):
    to_add=xkf1_m4_fault5[len(att_m4_fault5):]

for idx,i in enumerate(att_m4_fault5['Roll']):
    if(xkf1_m4_fault5['Roll'][idx] != i):
        att_m4_fault5['Roll'][idx] = ((att_m4_fault5['Roll'][idx] + xkf1_m4_fault5['Roll'][idx])/2)

for idx,i in enumerate(att_m4_fault5['Pitch']):
    if(xkf1_m4_fault5['Pitch'][idx] != i):
        att_m4_fault5['Pitch'][idx] = ((att_m4_fault5['Pitch'][idx] + xkf1_m4_fault5['Pitch'][idx])/2)

for idx,i in enumerate(att_m4_fault5['Yaw']):
    if(xkf1_m4_fault5['Yaw'][idx] != i):
        att_m4_fault5['Yaw'][idx] = ((att_m4_fault5['Yaw'][idx] + xkf1_m4_fault5['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m4_fault5=pd.concat([att_m4_fault5,to_add])

print(att_m4_fault5)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     121409547.0     0.29  1.06     -0.18   0.85   11.69  11.75
1     121423615.0     0.29  1.01     -0.19   0.87   11.69  11.74
2     121434103.0     0.29  1.10     -0.20   0.86   11.69  11.75
3     121444886.0     0.29  1.16     -0.20   0.87   11.69  11.74
4     121455563.0     0.29  1.13     -0.21   0.88   11.69  11.75
...           ...      ...   ...       ...    ...     ...    ...
5181  178891074.0     2.47 -0.12     -0.14  -0.02    6.91   8.32
5182  178902591.0     2.50 -0.23     -0.22  -0.05    6.91   8.15
5183  178912882.0     2.54 -0.31     -0.32   0.00    6.90   8.00
5184  178922946.0     2.58 -0.40     -0.42   0.11    6.90   7.82
5185  178932994.0     2.62 -0.43     -0.51   0.34    6.90   7.65

[5186 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m4_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m4_fault5 = esc_0_m4_fault5.astype("float64")
esc_0_m4_fault5 = esc_0_m4_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m4_fault5 = esc_0_m4_fault5[((esc_0_m4_fault5['TimeUS'] >= min) & (esc_0_m4_fault5['TimeUS'] <= max))]
esc_0_m4_fault5=esc_0_m4_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m4_fault5 = pd.read_csv("ESC_1.csv")
esc_1_m4_fault5 = esc_1_m4_fault5.astype("float64")
esc_1_m4_fault5 = esc_1_m4_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m4_fault5 = esc_1_m4_fault5[((esc_1_m4_fault5['TimeUS'] >= min) & (esc_1_m4_fault5['TimeUS'] <= max))]
esc_1_m4_fault5=esc_1_m4_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m4_fault5 = pd.read_csv("ESC_2.csv")
esc_2_m4_fault5 = esc_2_m4_fault5.astype("float64")
esc_2_m4_fault5 = esc_2_m4_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m4_fault5 = esc_2_m4_fault5[((esc_2_m4_fault5['TimeUS'] >= min) & (esc_2_m4_fault5['TimeUS'] <= max))]
esc_2_m4_fault5=esc_2_m4_fault5.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m4_fault5 = pd.read_csv("ESC_3.csv")
esc_3_m4_fault5 = esc_3_m4_fault5.astype("float64")
esc_3_m4_fault5 = esc_3_m4_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m4_fault5 = esc_3_m4_fault5[((esc_3_m4_fault5['TimeUS'] >= min) & (esc_3_m4_fault5['TimeUS'] <= max))]
esc_3_m4_fault5=esc_3_m4_fault5.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m4_fault5 = pd.read_csv("ESC_4.csv")
esc_4_m4_fault5 = esc_4_m4_fault5.astype("float64")
esc_4_m4_fault5 = esc_4_m4_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m4_fault5 = esc_4_m4_fault5[((esc_4_m4_fault5['TimeUS'] >= min) & (esc_4_m4_fault5['TimeUS'] <= max))]
esc_4_m4_fault5=esc_4_m4_fault5.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m4_fault5 = pd.read_csv("ESC_5.csv")
esc_5_m4_fault5 = esc_5_m4_fault5.astype("float64")
esc_5_m4_fault5 = esc_5_m4_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m4_fault5 = esc_5_m4_fault5[((esc_5_m4_fault5['TimeUS'] >= min) & (esc_5_m4_fault5['TimeUS'] <= max))]
esc_5_m4_fault5=esc_5_m4_fault5.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [11]:
#valori di accelerazione e del giroscopio
imu_0_m4_fault5 = pd.read_csv("IMU_0.csv")
imu_0_m4_fault5 = imu_0_m4_fault5.astype("float64")
imu_0_m4_fault5 = imu_0_m4_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m4_fault5 = imu_0_m4_fault5[((imu_0_m4_fault5['TimeUS'] >= min) & (imu_0_m4_fault5['TimeUS'] <= max))]

imu_1_m4_fault5 = pd.read_csv("IMU_1.csv")
imu_1_m4_fault5 = imu_1_m4_fault5.astype("float64")
imu_1_m4_fault5 = imu_1_m4_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m4_fault5 = imu_1_m4_fault5[((imu_1_m4_fault5['TimeUS'] >= min) & (imu_1_m4_fault5['TimeUS'] <= max))]

imu_2_m4_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m4_fault5 = imu_2_m4_fault5.astype("float64")
imu_2_m4_fault5 = imu_2_m4_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m4_fault5 = imu_2_m4_fault5[((imu_2_m4_fault5['TimeUS'] >= min) & (imu_2_m4_fault5['TimeUS'] <= max))]

imu_m4_fault5 = pd.concat((imu_0_m4_fault5, imu_1_m4_fault5, imu_2_m4_fault5))
imu_m4_fault5=imu_m4_fault5.groupby(imu_m4_fault5.TimeUS, as_index=False).mean()

display(imu_m4_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,121398893.0,0.132808,0.013634,-0.021743,-0.066052,0.333366,-10.232130
1,121401322.0,0.134899,0.042302,-0.007560,-0.054088,0.239980,-10.417303
2,121404530.0,0.053924,0.051608,0.004378,-0.022613,0.177931,-10.366177
3,121407516.0,0.035198,0.042260,0.006708,-0.012213,0.160736,-10.143430
4,121409376.0,0.102567,0.003226,-0.004532,0.010623,0.120311,-10.061591
...,...,...,...,...,...,...,...
19700,178922699.0,-0.208411,0.186337,-0.306661,0.568383,-0.142049,-10.251533
19701,178925360.0,-0.282195,0.364126,-0.281562,0.484639,0.045756,-10.359793
19702,178928239.0,-0.147285,0.446044,-0.268782,0.720112,0.264529,-10.229947
19703,178930888.0,-0.017998,0.382755,-0.279493,0.922468,0.259971,-10.184590


In [12]:
#sincronizzazione dei tempi tra att e imu_2
m4_fault5 = pd.merge_ordered(imu_m4_fault5,att_m4_fault5)
m4_fault5=m4_fault5.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m4_fault5 = pd.merge_ordered(m4_fault5, esc_0_m4_fault5)
m4_fault5 = pd.merge_ordered(m4_fault5, esc_1_m4_fault5)
m4_fault5 = pd.merge_ordered(m4_fault5, esc_2_m4_fault5)
m4_fault5 = pd.merge_ordered(m4_fault5, esc_3_m4_fault5)
m4_fault5 = pd.merge_ordered(m4_fault5, esc_4_m4_fault5)
m4_fault5 = pd.merge_ordered(m4_fault5, esc_5_m4_fault5)
m4_fault5=m4_fault5.fillna(method="ffill").fillna(method="bfill")
m4_fault5 = pd.merge_ordered(m4_fault5, rcou_m4_fault5)
m4_fault5=m4_fault5.fillna(method="ffill").fillna(method="bfill")

In [13]:
m4_fault5["TimeUS"] = m4_fault5["TimeUS"] - m4_fault5.iloc[0]["TimeUS"]
m4_fault5["TimeUS"] = m4_fault5["TimeUS"].astype(int)
m4_fault5["Guasto"] = guasto

In [14]:
last_cell = m4_fault5.iloc[-1, m4_fault5.columns.get_loc('TimeUS')]
print(last_cell)

57536491


In [15]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [16]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
20134  57522838
20135  57525695
20136  57528552
20137  57531409
20138  57534266

[20139 rows x 1 columns]


In [17]:
m4_fault5_final = pd.merge_ordered(m4_fault5,df)
m4_fault5_final=m4_fault5_final.fillna(method="ffill").fillna(method="bfill")
print(m4_fault5_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0  0.132808  0.013634 -0.021743 -0.066052  0.333366 -10.232130   
1          1556  0.132808  0.013634 -0.021743 -0.066052  0.333366 -10.232130   
2          2857  0.132808  0.013634 -0.021743 -0.066052  0.333366 -10.232130   
3          3985  0.134899  0.042302 -0.007560 -0.054088  0.239980 -10.417303   
4          5714  0.134899  0.042302 -0.007560 -0.054088  0.239980 -10.417303   
...         ...       ...       ...       ...       ...       ...        ...   
52723  57533551 -0.017998  0.382755 -0.279493  0.922468  0.259971 -10.184590   
52724  57534266 -0.017998  0.382755 -0.279493  0.922468  0.259971 -10.184590   
52725  57535553 -0.073281  0.372052 -0.289674  0.626604  0.254893 -10.443483   
52726  57535657 -0.073281  0.372052 -0.289674  0.626604  0.254893 -10.443483   
52727  57536491 -0.073281  0.372052 -0.289674  0.626604  0.254893 -10.443483   

       DesRoll  Roll  DesPitch  ...  Cu

In [18]:
m4_fault5_final = m4_fault5_final[m4_fault5_final.TimeUS.isin(to_be)]

In [19]:
m4_fault5_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,0,0.132808,0.013634,-0.021743,-0.066052,0.333366,-10.232130,0.29,1.06,-0.18,...,1.93,3579.0,2.37,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0,1.0
2,2857,0.132808,0.013634,-0.021743,-0.066052,0.333366,-10.232130,0.29,1.06,-0.18,...,1.93,3579.0,2.37,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0,1.0
4,5714,0.134899,0.042302,-0.007560,-0.054088,0.239980,-10.417303,0.29,1.06,-0.18,...,1.93,3579.0,2.37,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0,1.0
6,8571,0.053924,0.051608,0.004378,-0.022613,0.177931,-10.366177,0.29,1.06,-0.18,...,1.93,3579.0,2.37,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0,1.0
8,11428,0.035198,0.042260,0.006708,-0.012213,0.160736,-10.143430,0.29,1.06,-0.18,...,1.93,3579.0,2.37,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52713,57522838,-0.085935,0.221196,-0.261282,0.674822,0.129911,-9.815196,2.54,-0.31,-0.32,...,2.12,6050.0,3.02,1514.0,1468.0,1544.0,1435.0,1355.0,1604.0,1.0
52717,57525695,-0.208411,0.186337,-0.306661,0.568383,-0.142049,-10.251533,2.58,-0.40,-0.42,...,2.12,6050.0,3.02,1514.0,1468.0,1544.0,1435.0,1355.0,1604.0,1.0
52719,57528552,-0.282195,0.364126,-0.281562,0.484639,0.045756,-10.359793,2.58,-0.40,-0.42,...,2.12,6050.0,3.02,1514.0,1468.0,1544.0,1435.0,1355.0,1604.0,1.0
52722,57531409,-0.147285,0.446044,-0.268782,0.720112,0.264529,-10.229947,2.58,-0.40,-0.42,...,2.12,6050.0,3.02,1514.0,1468.0,1544.0,1435.0,1355.0,1604.0,1.0


In [20]:
from datetime import timedelta
m4_fault5_final=m4_fault5_final.reset_index(drop=True)
m4_fault5_final['TimeUS'] = pd.to_datetime(m4_fault5_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m4_fault5_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.132808,0.013634,-0.021743,-0.066052,0.333366,-10.232130,0.29,1.06,-0.18,...,1.93,3579.0,2.37,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0,1.0
1,00:00:00.002857,0.132808,0.013634,-0.021743,-0.066052,0.333366,-10.232130,0.29,1.06,-0.18,...,1.93,3579.0,2.37,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0,1.0
2,00:00:00.005714,0.134899,0.042302,-0.007560,-0.054088,0.239980,-10.417303,0.29,1.06,-0.18,...,1.93,3579.0,2.37,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0,1.0
3,00:00:00.008571,0.053924,0.051608,0.004378,-0.022613,0.177931,-10.366177,0.29,1.06,-0.18,...,1.93,3579.0,2.37,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0,1.0
4,00:00:00.011428,0.035198,0.042260,0.006708,-0.012213,0.160736,-10.143430,0.29,1.06,-0.18,...,1.93,3579.0,2.37,1422.0,1351.0,1373.0,1402.0,1407.0,1368.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20134,00:00:57.522838,-0.085935,0.221196,-0.261282,0.674822,0.129911,-9.815196,2.54,-0.31,-0.32,...,2.12,6050.0,3.02,1514.0,1468.0,1544.0,1435.0,1355.0,1604.0,1.0
20135,00:00:57.525695,-0.208411,0.186337,-0.306661,0.568383,-0.142049,-10.251533,2.58,-0.40,-0.42,...,2.12,6050.0,3.02,1514.0,1468.0,1544.0,1435.0,1355.0,1604.0,1.0
20136,00:00:57.528552,-0.282195,0.364126,-0.281562,0.484639,0.045756,-10.359793,2.58,-0.40,-0.42,...,2.12,6050.0,3.02,1514.0,1468.0,1544.0,1435.0,1355.0,1604.0,1.0
20137,00:00:57.531409,-0.147285,0.446044,-0.268782,0.720112,0.264529,-10.229947,2.58,-0.40,-0.42,...,2.12,6050.0,3.02,1514.0,1468.0,1544.0,1435.0,1355.0,1604.0,1.0


## Analisi nel tempo e in frequenza

In [21]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [23]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m4_fault5_final)/350)):
        V1=m4_fault5_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

In [24]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_29256\3329109307.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_29256\3329109307.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.118108,0.040139,-1.530838,0.232323,66.0,84.0,5.323947,3.480476,0.018595,...,7899.862215,4.057578,0.792591,-0.408120,4.153827,111.0,109.0,26.307887,9.719044,1
1,1.0,-0.118919,0.040256,-1.532474,0.232987,61.0,80.0,5.292230,3.490611,0.019167,...,8324.365408,4.059801,0.792072,-0.402505,4.155936,111.0,109.0,27.019137,10.806823,1
2,2.0,-0.119719,0.040367,-1.534154,0.233635,66.0,80.0,4.791316,3.509652,0.019626,...,8728.628245,4.062011,0.791546,-0.396854,4.158033,113.0,111.0,27.716136,11.864249,1
3,3.0,-0.120510,0.040474,-1.535850,0.234270,62.0,76.0,4.695250,3.660103,0.019976,...,9107.312779,4.064209,0.791014,-0.391166,4.160117,109.0,107.0,28.398417,12.881153,1
4,4.0,-0.121424,0.040657,-1.534536,0.235130,68.0,82.0,5.165157,3.684914,0.020472,...,9455.145429,4.066394,0.790475,-0.385443,4.162188,109.0,107.0,29.065549,13.847100,1
5,5.0,-0.122246,0.040782,-1.535637,0.235821,60.0,74.0,5.423601,3.653394,0.021045,...,9766.989337,4.068567,0.789929,-0.379684,4.164247,115.0,113.0,29.717137,14.751657,1
6,6.0,-0.122864,0.040804,-1.537516,0.236190,58.0,70.0,5.053047,3.597645,0.021323,...,10037.916135,4.070728,0.789377,-0.373891,4.166293,111.0,109.0,30.352827,15.584657,1
7,7.0,-0.123546,0.040856,-1.539533,0.236655,64.0,82.0,4.504661,3.469555,0.021529,...,10263.275818,4.072877,0.788819,-0.368063,4.168326,109.0,107.0,30.972298,16.336476,1
8,8.0,-0.124467,0.041047,-1.537391,0.237539,65.0,82.0,4.881678,3.389986,0.022050,...,10438.763441,4.075014,0.788255,-0.362201,4.170347,111.0,109.0,31.575263,16.998290,1
9,9.0,-0.125384,0.041235,-1.535433,0.238415,62.0,81.0,5.471633,3.307713,0.022568,...,10560.481440,4.077139,0.787685,-0.356306,4.172356,109.0,107.0,32.161471,17.562318,1


In [25]:
path_file = path_file.replace(r"M4\FAULT_M4_5", "")
os.chdir(path_file)
df_merged.to_csv('m4_fault5.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
